In [30]:
import ROOT as r
import os
import sys

In [31]:
filePath = "./data/"
fileName = "gm2skim_ana_16495.root"
station = 13

outfileName = fileName.split(".root")[0]+"_output.root"
f = r.TFile(filePath+fileName)

f.ls()

TFile**		./data/gm2skim_ana_16495.root	
 TFile*		./data/gm2skim_ana_16495.root	
  KEY: TDirectoryFile	clusterTree;1	clusterTree (ClusterAndCrystalHitTree) folder
  KEY: TDirectoryFile	testCoincidenceFinder;1	testCoincidenceFinder (TestCoincidenceFinderLM) folder
  KEY: TDirectoryFile	trackerNTup;1	trackerNTup (TrackNTup) folder
  KEY: TDirectoryFile	allmuons;1	allmuons (allmuons) folder
  KEY: TDirectoryFile	lostmuons;1	lostmuons (lostmuons) folder


# Plots to make
* All Cluster y's vs. Time
* All Tracks y's (calo face) vs. y-s (beam) vs. time 
* All calo-matched tracks y's (cluster) vs. y's (calo face tracker) vs. time

---

### Global Parameters

---

In [32]:
eMin = 1100
eMax = 2400
tMin = 0
tMax = 700
tBins = 700


histsToSave = []

### All cluster y's vs. time

---

In [33]:
cutString = "energy > "+str(eMin)+" && energy < "+str(eMax)+" && time/1000.*1.25 < "+str(tMax)+" && time/1000.*1.25 > "+str(tMin)+" && caloNum == "+str(station)
print(cutString)

energy > 1100 && energy < 2400 && time/1000.*1.25 < 700 && time/1000.*1.25 > 0 && caloNum == 13


In [34]:
t = f.Get("clusterTree").Get("clusters")
t.Print()

******************************************************************************
*Tree    :clusters  : clusters                                               *
*Entries :  2486009 : Total =       441720010 bytes  File  Size =  247610880 *
*        :          : Tree compression factor =   1.78                       *
******************************************************************************
*Br    0 :energy    : energy/D                                               *
*Entries :  2486009 : Total  Size=   19940672 bytes  File Size  =   18934102 *
*Baskets :      539 : Basket Size=    4585984 bytes  Compression=   1.05     *
*............................................................................*
*Br    1 :time      : time/D                                                 *
*Entries :  2486009 : Total  Size=   19939586 bytes  File Size  =   19000127 *
*Baskets :      539 : Basket Size=    4585984 bytes  Compression=   1.05     *
*...................................................

In [35]:
h1 = r.TH2I("allClusterY_vs_Time","Cluster y [mm] vs. Time; Time [#mus]; Relative y [mm]",
           tBins, tMin, tMax,
           600,-75,75)

t.Draw("y:time*1.25/1000.>>allClusterY_vs_Time",cutString, "goff")

39858

In [36]:
histsToSave.append(h1)

---

### All Tracks y's (calo face) vs. y-s (beam) vs. time 

---

In [37]:
t = f.Get("trackerNTup").Get("tracker")
t.Print()

******************************************************************************
*Tree    :tracker   : tracker name                                           *
*Entries :   100906 : Total =        17435182 bytes  File  Size =   12419575 *
*        :          : Tree compression factor =   1.40                       *
******************************************************************************
*Br    0 :runNum    : runNum/I                                               *
*Entries :   100906 : Total  Size=     405277 bytes  File Size  =       3553 *
*Baskets :       14 : Basket Size=      32000 bytes  Compression= 113.90     *
*............................................................................*
*Br    1 :subRunNum : subRunNum/I                                            *
*Entries :   100906 : Total  Size=     405331 bytes  File Size  =       3570 *
*Baskets :       14 : Basket Size=      32000 bytes  Compression= 113.37     *
*...................................................

In [38]:
cutString = "passVertexQuality && passTrackQuality && station == "+str(station-1)

In [39]:
h2 = r.TH3I("allTracks_yCalo_vs_yBeam_vs_Time", 'Track y vs. Extrapolated Calo y vs. Time; Time [#mus]; Track y [mm]; Calo y [mm]',
           tBins, tMin, tMax,
            400, -50, 50,
            600, -100,100
           )

t.Draw("caloVertexPosY:decayVertexPosY:decayTime/1000.>>allTracks_yCalo_vs_yBeam_vs_Time", cutString, "goff")

3249

In [40]:
histsToSave.append(h2)

---

### All calo-matched tracks y's (cluster) vs. y's (calo face tracker) vs. time

---

In [41]:
t = f.Get("allmuons").Get("tree")
t.Print()

******************************************************************************
*Tree    :tree      : allmuons                                               *
*Entries :      675 : Total =        11901520 bytes  File  Size =    5980038 *
*        :          : Tree compression factor =   1.99                       *
******************************************************************************
*Br    0 :midasSerialNum : midasSerialNum/i                                  *
*Entries :      675 : Total  Size=       3788 bytes  File Size  =       1922 *
*Baskets :        7 : Basket Size=      32000 bytes  Compression=   1.70     *
*............................................................................*
*Br    1 :runNum    : runNum/i                                               *
*Entries :      675 : Total  Size=       3700 bytes  File Size  =        693 *
*Baskets :        7 : Basket Size=      32000 bytes  Compression=   4.63     *
*...................................................

In [42]:
cutString = "trkPassCaloVertexQuality && trkPassTrackQuality && trkPassVertexQuality " 
cutString += " && cluEne > "+str(eMin)+" && cluEne < "+str(eMax)
cutString += " && cluTime/1000. < "+str(tMax)+" && cluTime/1000. > "+str(tMin)
cutString += " && cluCaloNum == "+str(station)

print(cutString)

trkPassCaloVertexQuality && trkPassTrackQuality && trkPassVertexQuality  && cluEne > 1100 && cluEne < 2400 && cluTime/1000. < 700 && cluTime/1000. > 0 && cluCaloNum == 13


In [43]:
h3 = r.TH3I("matchedTracks_yCaloTracks_vs_yCaloTracker_vs_Time", 
            'Tracker Calo y vs. Cluster Calo y vs. Time; Time [#mus]; Track y [mm]; Calo y [mm]',
            tBins, tMin, tMax,
            400, -50, 50,
            600, -100,100
           )

t.Draw("(cluY-3)*25:vY:cluTime/1000.>>matchedTracks_yCaloTracks_vs_yCaloTracker_vs_Time", cutString, "goff")

848

In [44]:
histsToSave.append(h3)

---

### Save histograms to root file

---

In [45]:
fout = r.TFile(outfileName,"RECREATE")
for hi in histsToSave:
    hi.Write()
fout.Close()

In [46]:
f = r.TFile(outfileName)
f.ls()

TFile**		gm2skim_ana_16495_output.root	
 TFile*		gm2skim_ana_16495_output.root	
  KEY: TH2I	allClusterY_vs_Time;1	Cluster y [mm] vs. Time
  KEY: TH3I	allTracks_yCalo_vs_yBeam_vs_Time;1	Track y vs. Extrapolated Calo y vs. Time
  KEY: TH3I	matchedTracks_yCaloTracks_vs_yCaloTracker_vs_Time;1	Tracker Calo y vs. Cluster Calo y vs. Time
